# Step3

任意の音源にモデルを適応させてみるフェーズ

とりあえずテストなので最初の２０秒だけ出力します

In [93]:
import step_utils as ut
import model as mm
from parameter import Parameter

import re
import librosa
import numpy as np
import torch
from IPython.display import display,Audio

In [157]:
audio_path = "./datasets/test2.wav"
model_path = "model_layer4_20201130_143624_Epoch30.pt"

In [ ]:
feat = 258
sequence = 513

p=Parameter()
sample_rate = p.sample_rate
audio_len = p.audio_len

In [167]:
def test_listen(audio_path,model_path,sample_rate,audio_len,feat,sequence):
    data, _  = librosa.load(audio_path, sr=sample_rate)
    t_data=torch.from_numpy(data.astype(np.float32)).clone()
    stftdata = torch.stft(t_data,2**8,return_complex=True)
    realpart,imagpart = stftdata.real,stftdata.imag
    addnoise = torch.cat((realpart,imagpart),0)
    num_layer = int(re.sub("\\D", "", model_path)[0])
    model = mm.Net(sequence, feat, num_layer)
    q, mod = divmod(addnoise.shape[1], sequence)
    
    ret = torch.zeros([10,addnoise.shape[0],sequence])
    if q < 10:
        for i in range(q):
            a = addnoise[:,i*sequence : (i+1)*sequence]
            ret[i,:,:] = a
    else:
        for i in range(10):
            a = addnoise[:,i*sequence : (i+1)*sequence]
            ret[i,:,:] = a
    model.eval()
    
    mask = model(ret.float())
    h_hat = mask * ret
    for i in range(10):
        if i== 0:   
            audio = ut.tensorstft2audio(h_hat[i,:,:])

        else:
            x = ut.tensorstft2audio(h_hat[i,:,:])
            audio = np.concatenate([audio, x])
    print("Original")
    display(Audio(data,rate = sample_rate))
    print("Test")
    display(Audio(audio,rate = sample_rate))

In [168]:
test_listen(audio_path,model_path,sample_rate,audio_len,feat,sequence)

Original


Test
